- https://lilianweng.github.io/posts/2021-05-31-contrastive/
- siamese-triplet
    - https://github.com/adambielski/siamese-triplet/tree/master
- contrastive-image-text
    - CLIP：https://github.com/huggingface/transformers/tree/main/examples/pytorch/contrastive-image-text

### constrastive learning & metric learning 

- constrastive learning
    - 一种通过对样本对进行比较来学习有用表征的方法。目标是使相似样本的表示更接近，不相似样本的表示更远离。
    - 相似不相似，更多是类别上的；
- metric learning or embedding learning
    - embedding learning：学习的是表示；
        - 传统对比学习主要关注学习数据的表示（embedding），使得相似样本的表示更接近，以便在下游任务中使用。
- learning to rank（learning from preference）
    - 从类别到偏序
- 数据集（样本对，正例、负例，或者偏序对）的构造，以及损失函数的设计；

## constrastive loss

### (pairwise) Margin Ranking loss

$$
L(y_w,y_l)=\max(0, \delta-(R(y_w) - R(y_l))
$$

- 也就是说，目标追求的是 $R(y_w)-R(y_l)\geq \delta$ 时，loss 才为0；

    $$
    R(y_w)-R(y_l)\geq \delta\\
    R(y_w)\geq R(y_l)+\delta
    $$
    
    - Reward model 对 $y_w$ 的打分要比对 $y_l$ 的打分 + $\delta$ 还多；